### **4.2 - Sharing data with XCOMs and limitations**

Como sabrá, los XComs son un mecanismo de Airflow que le permite compartir datos entre sus tareas. Básicamente, si quieres compartir un valor como un string o algún dato, puedes ponerlo en un Xcom y este XCom es un objeto que será almacenado dentro de tu meta database con la key y el valor que quieres almacenar. Entonces podrás extraer (pull) ese XCom de otra tarea para que puedas compartir datos entre tus tareas. Es tan simple como eso, pero hay algunas limitaciones y tienes que ser consciente de ellas, de lo contrario terminarás con algunos problemas. Comencemos con las limitaciones y luego las diferentes formas de obtener tus XComs y la última forma es mi favorita. Empecemos. En tu data pipeline tienes dos tareas: "extract" y "process":

<center><img src="https://i.postimg.cc/prNdMmYB/a1162.png"></center>

y quieres compartir datos entre esas dos tareas, ¿cómo puedes hacerlo? Utilizando XComs. Imaginemos que tienes un valor en "extract", digamos, partner_name="netflix", digamos que netflix es tu partner y entonces quieres compartir estos datos con tu tarea "process". Para hacer eso, necesitas crear tu XCom, y para crear ese XCom necesitas acceder al task instance object. Así que al utilizar el Python operator para acceder al task instance object sólo tienes que escribir "ti":

<center><img src="https://i.postimg.cc/Y2g9fxMD/a1163.png"></center>

Y si te preguntas qué es el task instance object, pues básicamente cada vez que disparas tu tarea en Airflow esa tarea se convierte en un task instance. Y con su task instance object, puede acceder al contexto de la tarea y al mecanismo XCom. Así que una vez que tienes "ti", puedes escribir "ti.xcom_push" y luego la key, así que llamémosla "partner_name" con el valor que quieres compartir, en ese caso es "partner_name". Es tan sencillo como eso, pero ten en cuenta que por defecto tu XCom está serializado, el valor de ahí mismo está serializado:

<center><img src="https://i.postimg.cc/wMhjq9wY/a1164.png"></center>

como un valor Jason. Antes de Airflow 2.0, este valor fue decapado (pickled), pero ya no es el caso por razones de seguridad. Así que una vez que haya empujado (pushed) sus XComs, será capaz de regresar de la otra tarea (getting back from the other task) simplemente usando el task instance object con "ti" y el tipo "partner_name = ti.xcom_pull(entonces debe especificar la key de su XCom), en ese caso es "key=partner_name" y los task IDs, más específicamente el task ID donde ese XCom ha sido empujado (pushed), en ese caso es "extract". 

<center><img src="https://i.postimg.cc/RVB0Pn0S/a1165.png"></center>

Una vez que tienes eso, guarda el archivo y vamos a verificar si funciona. Así que escriba "docker ps" en su terminal y escriba "docker exec -it (vamos a utilizar el ID del contenedor del scheduler) /bin/bash") con el fin de acceder a la Airflow CLI y, a continuación, escriba "airflow tasks test (el DAG ID "my_dag") y luego "extract" y una fecha de ejecución en el pasado". Pulse Enter. 

<center><img src="https://i.postimg.cc/yYy82YJF/a1166.png"></center>

Esperemos un poco. Ok, como puedes ver funciona. 

<center><img src="https://i.postimg.cc/FK8r0YqQ/a1167.png"></center>

Ahora verifiquemos que somos capaces de acceder al valor "partner_name" desde el proceso, así que escribamos "print(partner_name)" y guardemos el archivo de nuevo y ejecutemos la tarea "process". Pulsamos enter. Esperemos un poco:

<center><img src="https://i.postimg.cc/2SjSYRvg/a1168.png"></center>

Y como puedes ver obtienes "netflix". 

<center><img src="https://i.postimg.cc/GpH3BFXQ/a1169.png"></center>

Bien, así es como puedes compartir datos entre tus tareas. Ahora, antes de pasar a las diferentes formas de manipular tus XComs, vamos a hablar de las limitaciones, porque es muy fácil pensar que, ¡eh!, tengo un dataframe, quiero compartir ese datafame entre mis diferentes tareas, así que ¿por qué no usar XComs? Bueno, si tratas de hacer eso, y tu dataframe es demasiado grande, terminarás con problemas. Así que lo primero que tienes que saber es que los XComs tienen un tamaño limitado. ¿Qué tamaño? Esto depende de tu meta database. Con SQLite estás limitado a 2 gigabytes para un XCom determinado. Con Postgres estás limitado a 1 GB para un XCom dado y si te basas en MySQL entonces eres capaz de almacenar como máximo 64 kilobytes en un XCom dado. Por eso tienes que tener mucho cuidado con el tamaño de los datos que compartes entre tus tareas y, de hecho, como mejor práctica deberías usar XComs sólo para compartir una pequeña cantidad de datos. Y ahora te veo venir, ¡hey! hay esta nueva característica XCom backend, así que soy libre de, almacenar tantos datos como quiera y compartirlos entre mis tareas. En realidad, no, si usted hace eso, de nuevo, usted terminará con un "error de desbordamiento de memoria (Memory overflow error)", porque al final del día, usted va a tirar de la XCom de su backend dentro de su tarea, dentro de su Airflow cluster. Así que, si usted utiliza un XCom backend, para decir, Ok, en lugar de almacenar mi XCom dentro de la meta database, quiero almacenarlo en S3. Usted tendrá que lidiar con las referencias a sus datos y no sus datos en sí. De lo contrario, usted terminará con el "error de desbordamiento de memoria". Ok, así que incluso con los backends de XCom, usted todavía tiene que tener cuidado con las limitaciones en términos de tamaño. 

Muy bien, es hora de hablar de las diferentes formas de empujar tu XCom en Airflow, y como vas a ver es bastante interesante y la última forma es definitivamente mi favorita. Así que vamos a empezar con la primera. Aquí eres capaz de empujar (push) el XCom con "xcom_push" accediendo al task instance object, bastante simple. Pero, usted puede hacer aún más simple que eso. Usted puede simplemente devolver un valor, así, por ejemplo, "return partner_name" como usted quiere compartir partner name y al hacer esto usted va a empujar XCom con el valor "partner_name" y la key "return_value", que es la key por defecto, cada vez que se crea XCom así :

<center><img src="https://i.postimg.cc/HkSssZ9n/a1170.png"></center>

Ok, entonces es absolutamente lo mismo que antes, pero esta vez la key es "return_value". Así que para recuperar este XCom de la meta database en la tarea "process" tienes que modificar la key con "return_value":

<center><img src="https://i.postimg.cc/Y9FpRpKL/a1171.png"></center>

o simplemente eliminar esa key y usar solo el task ID. Y eso es todo. 

<center><img src="https://i.postimg.cc/qqdpZ94b/a1172.png"></center>

Podrás recuperar ese XCom. Así que esta es la segunda manera, ahora, ¿qué pasa si usted tiene múltiples Xcoms, múltiples valores que desea compartir entre sus tareas, por ejemplo, además de partner_name, digamos, usted tiene:

<center><img src="https://i.postimg.cc/VvBzyDNY/a1173.png"></center>

Bueno, no vas a hacer eso, porque crearás una conexión para cada XCom y es realmente verboso, así que definitivamente hay una mejor manera de empujar múltiples XComs a la vez y esa manera es devolviendo un "JSON value" Así que lo único que necesitas hacer aquí en eso, en lugar de devolver sólo "partner_name", creas un diccionario y luego pones:

<center><img src="https://i.postimg.cc/GhdcPwM9/a1174.png"></center>

Y haciendo esto empujas el XCom con el JSON value y así múltiples valores dentro de ese XCom, lo cual es definitivamente mucho más optimizado. Para recuperar ese XCom, sigue funcionando como antes, pero ahora "partner_name" ya no es "partner_name". Es "partner_" ,digamos, "partner_settings" y entonces puedes acceder a esos ajustes simplemente escribiendo, por ejemplo, "partner_name":

<center><img src="https://i.postimg.cc/L6DmNC7D/a1175.png"></center>
<center><img src="https://i.postimg.cc/mg6BGcTF/a1176.png"></center>

No olvide definir las dependencias entre sus tareas, en ese caso quiere ejecutar primero "extract" y luego "process" luego, guarde el archivo 

<center><img src="https://i.postimg.cc/4x3sMvRg/a1177.png"></center>

y vaya a la interfaz de usuario de Airflow, haga clic en ADMIN > XComs y, como puede ver, todavía no tiene ningún XCom. 

<center><img src="https://i.postimg.cc/zDnzRXF9/a1178.png"></center>

Vuelva a DAGs, haga clic en su DAG "my_dag", y luego Graph View. Usted obtuvo el siguiente data pipeline. Vamos a encender el toggle con el fin de disparar, actualizar la página, vamos a esperar un poco. 

<center><img src="https://i.postimg.cc/KYkFcZYp/a1179.png"></center>

Ahora ya está hecho. Si vas a ADMIN > XComs, obtienes el siguiente XCom con la key "return_value" y el siguiente valor correspondiente a tu valor JSON con "partner_name" y "partner_path". Obviamente, si vuelves a la Graph View, haces clic en la tarea "process" y "log", obtienes "netflix":

<center><img src="https://i.postimg.cc/bv7PsKDy/a1180.png"></center>
<center><img src="https://i.postimg.cc/D0FKcLPR/a1181.png"></center>

Ahora la cosa es que si vuelves a los XComs, qué pasa si quieres tener dos XComs diferentes, uno para "partner_name" y otro para "partner_path". 

<center><img src="https://i.postimg.cc/V6X8MDnj/a1182.png"></center>

Como puedes hacer eso, sin usar "xcom_push" dos veces. Bueno, esta es la tercera forma y mi forma favorita usando el "Taskflow API" pero primero vamos a descubrir lo que es el "Taskflow API" .